In [139]:
from imports import *
from information_conditions import Information_Conditions
from base_ecopg import *
from helper_functions import *
from simulation_and_results_functions import *

In [140]:
np.set_printoptions(precision=4, suppress=True, linewidth=200)


In [141]:
def add_degraded_state_policies_both_state_and_action(strategy):
        
        strategy_propserous_and_degraded_state = strategy.copy()

        for i in [0, 2, 4, 6]:
            x = np.random.rand()
            strategy_propserous_and_degraded_state.insert(i, x)
        
        return strategy_propserous_and_degraded_state


def add_degraded_state_policies_only_state(strategy):
        
        strategy_propserous_and_degraded_state = strategy.copy()
        x = np.random.rand()
        strategy_propserous_and_degraded_state.insert(0, x)
        
        return strategy_propserous_and_degraded_state



In [142]:
def create_policy_from_strategy(agent_1_strategy, agent_2_strategy):


    agent_1_strategy = [[x, 1-x] for x in agent_1_strategy]
    agent_2_strategy = [[x, 1-x] for x in agent_2_strategy]

    policy = np.array([agent_1_strategy, agent_2_strategy])

    return policy

In [143]:
def calculate_avg_value_given_policy(policy, mae, mode):
    Vio = mae.Vio(policy)
    obsdist = mae.obsdist(policy)

    # if mode == 'both_state_and_action_information' or mode == 'only_state_information':
    #     Vio = Vio[:,1::2]
    #     obsdist = obsdist[:,1::2]
    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vio, [agents, states], obsdist , [agents, states], [agents])
    return avg_value_agent_1, avg_value_agent_2

In [144]:
def calculate_avg_value_given_policy_state_mae_ecopg(policy, mode):

    mae_ecopg =  create_mae_ecopg_for_given_mode_POstratAC(mode)
    Vis = mae_ecopg.Vis(policy)
    mae_ecopg.has_last_statdist = False
    mae_ecopg._last_statedist = None
    state_dist = mae_ecopg._numpyPs(policy)

    # print(Vis, "vis")
    # print(state_dist, "state dist")
    
    if mode == 'complete' or mode == 'ecological':
        Vis = Vis[:,1::2]
        state_dist = state_dist[1::2]
        state_dist = state_dist/np.sum(state_dist)
    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vis, [agents, states], state_dist , [states], [agents])
    return  avg_value_agent_1, avg_value_agent_2


def calculate_avg_value_given_policy_state_test(policy, mode, mae):

    Vis = mae.Vis(policy)
    state_dist = mae.Ps(policy)

    # print(Vis, "vis")
    # print(state_dist, "state dist")
    
    if mode == 'complete' or mode == 'ecological':
        Vis = Vis[:,1::2]
        state_dist = state_dist[1::2]
        state_dist = state_dist/np.sum(state_dist)
    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vis, [agents, states], state_dist , [states], [agents])
    return  avg_value_agent_1, avg_value_agent_2



In [145]:
def metgame_reward_matrix(strategy_set_p1, strategy_set_p2, mae, mode):
    p1_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))
    p2_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))

    for i, p1_strategy in enumerate(strategy_set_p1.values()):
        for j, p2_strategy in enumerate(strategy_set_p2.values()):

    
            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            value_p1, value_p2 = calculate_avg_value_given_policy(policy, mae, mode)
            p1_reward_matrix[i,j] = value_p1
            p2_reward_matrix[i,j] = value_p2

    return p1_reward_matrix, p2_reward_matrix



def metgame_reward_matrix_state_ecopg(strategy_set_p1, strategy_set_p2, mode):
    '''Vi estimate using vis and Ps instead of Vio and obsdist'''
    p1_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))
    p2_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))

    for i, p1_strategy in enumerate(strategy_set_p1.values()):
        for j, p2_strategy in enumerate(strategy_set_p2.values()):

            # print(p1_strategy, p2_strategy, "strat p1, p2")

            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            value_p1, value_p2 = calculate_avg_value_given_policy_state_mae_ecopg(policy, mode)

            # print(value_p1, value_p2 ,"value p1, p2")
            p1_reward_matrix[i,j] = value_p1
            p2_reward_matrix[i,j] = value_p2

    return p1_reward_matrix, p2_reward_matrix



def metgame_reward_matrix_state_test(strategy_set_p1, strategy_set_p2, mode, mae):
    '''Vi estimate using vis and Ps instead of Vio and obsdist'''
    p1_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))
    p2_reward_matrix = np.zeros((len(strategy_set_p1), len(strategy_set_p2)))

    for i, p1_strategy in enumerate(strategy_set_p1.values()):
        for j, p2_strategy in enumerate(strategy_set_p2.values()):

            # print(p1_strategy, p2_strategy, "strat p1, p2")

            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            value_p1, value_p2 = calculate_avg_value_given_policy_state_test(policy, mode, mae)

            # print(value_p1, value_p2 ,"value p1, p2")
            p1_reward_matrix[i,j] = value_p1
            p2_reward_matrix[i,j] = value_p2

    return p1_reward_matrix, p2_reward_matrix


In [146]:
def create_determinstic_strategies_set_for_both_players(mode):
    '''creates deterministic strategy sets for given mode '''

    mae_ecopg_for_evaluating_no_of_states = create_mae_ecopg_for_given_mode_POstratAC(mode)
    number_of_states = mae_ecopg_for_evaluating_no_of_states.Q
    
    if mode == 'none' or mode == 'social':
        determinstic_strategy_itertools = itertools.product([1 , 0], repeat = number_of_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_full = {str(np.round(strat)):strat for strat in determinisic_strategy_lists}

    else:
        number_of_prosperous_states = int(number_of_states/2)
        determinstic_strategy_itertools = itertools.product([1 , 0], repeat = number_of_prosperous_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_only_prosperous = {str(np.round(strat)):strat for strat in determinisic_strategy_lists}

        if mode == 'complete':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}
        elif mode == 'ecological':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_only_state(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}


    strategy_set_p1 = all_determinstic_strategy_dictionary_full
    strategy_set_p2 = all_determinstic_strategy_dictionary_full

    return strategy_set_p1, strategy_set_p2

In [147]:
def reward_matrix_to_excel(p1_reward_matrix, p2_reward_matrix, strategy_set_p1, strategy_set_p2):

    '''p1 rewards pg 1. p2 rewards pg 2. rows -> p1 strategies, cols -> p2 strategies in both cases '''
    p1_reward_matrix = pd.DataFrame(p1_reward_matrix, index = list(strategy_set_p1.keys()), columns= list(strategy_set_p2.keys()))
    p2_reward_matrix = pd.DataFrame(p2_reward_matrix, index = list(strategy_set_p1.keys()), columns= list(strategy_set_p2.keys()))

    with pd.ExcelWriter('reward_matrix_only_action_2.xlsx') as excel_file:
        p1_reward_matrix.to_excel(excel_file, sheet_name='p1', index=True, header=True)
        p2_reward_matrix.to_excel(excel_file, sheet_name='p2', index=True, header=True)

In [148]:
def calculate_pure_strategy_nash_equilibria(payoffs_row, payoffs_col):
    game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)
    result = pygambit.nash.enumpure_solve(game)
    return result.equilibria

In [149]:
def process_and_print_nash_equilibria_results(nash_equlibria_result, strategy_set_p1, strategy_set_p2):

    number_of_nash_equilbria = len(nash_equlibria_result)
    print("Total number of Nash ", number_of_nash_equilbria)

    '''better readable format of the strategies'''
    strategies_p1_array = np.array(list(strategy_set_p1.keys())) #just the list of strategies
    strategies_p2_array = np.array(list(strategy_set_p2.keys()))

    for eq in nash_equlibria_result:                       # each equilibrium profile
        strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]    #converting mixed strategis to float

        p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool) #boolean conversion - for strategy in pure nash probability =  1, for strategy not in pure nash, probability = 0
        p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
        # float_profile = np.array() # iterate over (strategy, prob

        print('P1:' , strategies_p1_array[p1_active_strategy])   #will only print the strategy found in nash
        print('P2:' , strategies_p2_array[p2_active_strategy])

        print('------')


'''


strategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))
strategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))

print(strategy_set_p1_only_action_array)

for eq in result.equilibria:                       # each equilibrium profile
    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]

    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)
    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
    # float_profile = np.array() # iterate over (strategy, prob

    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])
    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])

    print('------')




'''

"\n\n\nstrategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))\nstrategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))\n\nprint(strategy_set_p1_only_action_array)\n\nfor eq in result.equilibria:                       # each equilibrium profile\n    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]\n\n    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)\n    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)\n    # float_profile = np.array() # iterate over (strategy, prob\n\n    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])\n    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])\n\n    print('------')\n\n\n\n\n"

In [150]:
def get_pure_nash_equilibria_all_deterministic_strategies_mode(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players(mode)
    p1_reward_matrix, p2_reward_matrix = metgame_reward_matrix_state_ecopg(strategy_set_p1, strategy_set_p2, mode)

    print("p1: \n", p1_reward_matrix, "\n p2: \n", p1_reward_matrix)
    
    nash_equlibrium_result = calculate_pure_strategy_nash_equilibria(p1_reward_matrix, p2_reward_matrix)
    process_and_print_nash_equilibria_results(nash_equlibrium_result, strategy_set_p1, strategy_set_p2)

In [153]:
get_pure_nash_equilibria_all_deterministic_strategies_mode('none')

p1: 
 [[  50.     -298.0397]
 [-295.5892 -298.5374]] 
 p2: 
 [[  50.     -298.0397]
 [-295.5892 -298.5374]]
Total number of Nash  1
P1: ['[1]']
P2: ['[1]']
------


In [152]:



mode = 'only_action_history_information'

p1_reward_matrix_only_action, p2_reward_matrix_only_action = metgame_reward_matrix_state(strategy_set_p1_only_action, strategy_set_p2_only_action, mode)
       



payoffs_row =  p1_reward_matrix_only_action
# payoffs_row_modifed = payoffs_row - payoffs_row[0,0]

print(payoffs_row)

payoffs_col  = p2_reward_matrix_only_action

print(payoffs_col)
game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)



# Compute pure strategy Nash equilibria
result = pygambit.nash.enumpure_solve(game)
# result = pygambit.nash.enummixed_solve(game)


print(len(result.equilibria))
 
# for i in result.equilibria:
#     print(i)  
#     print("---")
strategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))
strategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))

print(strategy_set_p1_only_action)

for eq in result.equilibria:                       # each equilibrium profile
    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]

    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)
    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
    # float_profile = np.array() # iterate over (strategy, prob

    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])
    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])

    print('------')


NameError: name 'strategy_set_p1_only_action' is not defined

In [ ]:
'''No Information: Calculating Nash equilibria '''


determinstic_strategy_itertools_no_info = itertools.product([1,0], repeat = 1)

determinisic_strategy_lists_no_info = list(list(strat) for strat in determinstic_strategy_itertools_no_info)
all_determinstic_strategy_dictionary_no_info = {str(strat):strat for strat in determinisic_strategy_lists_no_info}
strategy_set_p1_no_info = all_determinstic_strategy_dictionary_no_info
strategy_set_p2_no_info = all_determinstic_strategy_dictionary_no_info

print(strategy_set_p1_no_info)

mode = 'none'
ecopg = BaseEcologicalPublicGood(qc = 0)

information_condition_instance = Information_Conditions(ecopg, mode=mode)
mae_ecopg = POstratAC(env=information_condition_instance, learning_rates=0.05, discount_factors= 0.98)

p1_reward_matrix_no_info, p2_reward_matrix_no_info = metgame_reward_matrix_state(strategy_set_p1_no_info, strategy_set_p2_no_info, mode)


p1_reward_matrix_only_action_df = pd.DataFrame(p1_reward_matrix_no_info, index = list(strategy_set_p1_no_info.keys()), columns= list(strategy_set_p1_no_info.keys()))
p2_reward_matrix_only_action_df = pd.DataFrame(p2_reward_matrix_no_info, index = list(strategy_set_p1_no_info.keys()), columns= list(strategy_set_p2_no_info.keys()))

with pd.ExcelWriter('reward_matrix_no_info.xlsx') as excel_file:
    p1_reward_matrix_only_action_df.to_excel(excel_file, sheet_name='p1_reward_matrix', index=True, header=True)
    p2_reward_matrix_only_action_df.to_excel(excel_file, sheet_name='p2_reward_matrix', index=True, header=True)

    
payoffs_row = p1_reward_matrix_no_info
print(payoffs_row)
payoffs_col  = p2_reward_matrix_no_info
print(payoffs_col)
game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)


# Compute pure strategy Nash equilibria
result = pygambit.nash.enumpure_solve(game)
# result = pygambit.nash.enummixed_solve(game)

print(len(result.equilibria))
 
for i in result.equilibria:
    print(i)  
    print("---")

{'[1]': [1], '[0]': [0]}
[[  50.     -298.0397]
 [-295.5892 -298.5374]]
[[  50.     -295.5892]
 [-298.0397 -298.5374]]
1
[[Rational(1, 1), Rational(0, 1)], [Rational(1, 1), Rational(0, 1)]]
---


In [ ]:
       # pygambit is imported as “gambit

payoffs_row =  p1_reward_matrix_only_action
payoffs_row_modifed = payoffs_row - payoffs_row[0,0]

print(payoffs_row_modifed)

payoffs_col  = np.transpose(p2_reward_matrix_only_action)

game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)



# Compute pure strategy Nash equilibria
result = pygambit.nash.enumpure_solve(game)
# result = pygambit.nash.enummixed_solve(game)


print(len(result.equilibria))
 
# for i in result.equilibria:
#     print(i)  
#     print("---")
strategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))
strategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))

print(strategy_set_p1_only_action_array)

for eq in result.equilibria:                       # each equilibrium profile
    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]

    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)
    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
    # float_profile = np.array() # iterate over (strategy, prob

    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])
    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])

    print('------')


[[  0.       0.9817   2.9482   2.9482   0.       0.9817   2.9482   2.9482   0.       0.9817   2.9482   2.9482   0.       0.9817   2.9482   2.9482]
 [  0.1647   1.7055   2.9482   2.9482   0.8622   1.7055   2.9482   2.9482   0.1647   2.1212   2.9482   2.9482   0.8622   2.1212   2.9482   2.9482]
 [  0.       0.8599   0.       2.1185   1.7226   1.7226   0.       2.1185   0.       0.8599   6.3225   6.3225   1.7226   1.7226   6.3225   6.3225]
 [  0.1647   1.7055   0.1647   1.7055   1.7226   1.7249   1.7152   1.7055   0.1647   1.7155   6.3282   6.3226   1.7226   1.7226   6.3225   6.3226]
 [  0.4977   0.5047   0.4977   0.4977   0.       0.9817   2.9482   2.9482   0.4977   0.5047   0.4977   0.4977   0.       0.9817   2.9482   2.9482]
 [  0.4977   0.4977   0.4977   2.5669   1.307    1.7055   2.948    2.9482   0.4977   0.8967   0.4977   0.4977   1.7189   2.1212   2.9482   2.9482]
 [  0.4977   0.4977   0.4977   0.8177   0.       1.7155   0.       2.1185   0.       0.4977   0.       6.3133   0.    

In [ ]:
strategy_set_p1_both_state_and_action = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in strategy_set_p1_only_action.items()}
strategy_set_p2_both_state_and_action = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in strategy_set_p2_only_action.items()}


mode = 'both_state_and_action_information'
ecopg = BaseEcologicalPublicGood()

information_condition_instance = Information_Conditions(ecopg, mode=mode)
mae_ecopg = POstratAC(env=information_condition_instance, learning_rates=0.05, discount_factors= 0.98)

p1_reward_matrix_both_state_and_action, p2_reward_matrix_both_state_and_action = metgame_reward_matrix_state(strategy_set_p1_both_state_and_action, strategy_set_p2_both_state_and_action, mae_ecopg, mode)
       

# p1_reward_matrix_both_state_and_action_df = pd.DataFrame(p1_reward_matrix_both_state_and_action, index = list(strategy_set_p1_both_state_and_action.keys()), columns= list(strategy_set_p2_both_state_and_action.keys()))
# p2_reward_matrix_both_state_and_action_df = pd.DataFrame(p2_reward_matrix_both_state_and_action, index = list(strategy_set_p2_both_state_and_action.keys()), columns= list(strategy_set_p2_both_state_and_action.keys()))

# with pd.ExcelWriter('reward_matrix_both_state_and_action.xlsx') as excel_file:
#     p1_reward_matrix_both_state_and_action_df.to_excel(excel_file, sheet_name='p1_reward_matrix', index=True, header=True)
#     p2_reward_matrix_both_state_and_action_df.to_excel(excel_file, sheet_name='p2_reward_matrix', index=True, header=True)

In [ ]:
       # pygambit is imported as “gambit

payoffs_row =  p1_reward_matrix_both_state_and_action
payoffs_row_modifed = payoffs_row - payoffs_row[0,0]

print(payoffs_row_modifed)

payoffs_col  = p2_reward_matrix_both_state_and_action

game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)



# Compute pure strategy Nash equilibria
result = pygambit.nash.enumpure_solve(game)
# result = pygambit.nash.enummixed_solve(game)


print(len(result.equilibria))
 
# for i in result.equilibria:
#     print(i)  
#     print("---")
strategy_set_p1_only_action_array = np.array(list(strategy_set_p1_only_action.keys()))
strategy_set_p2_only_action_array = np.array(list(strategy_set_p2_only_action.keys()))

print(strategy_set_p1_only_action_array)

for eq in result.equilibria:                       # each equilibrium profile
    strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]

    p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool)
    p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
    # float_profile = np.array() # iterate over (strategy, prob

    print('P1:' , strategy_set_p1_only_action_array[p1_active_strategy])
    print('P2:' , strategy_set_p2_only_action_array[p2_active_strategy])

    print('------')


[[   0.        0.        0.        0.        0.        0.        0.        0.     -129.9658 -129.9729 -129.9667 -129.9738 -216.8567 -216.8956 -216.8618 -216.9008]
 [   0.        0.       -0.       -0.        0.        0.        0.        0.     -129.9498 -163.5421 -129.9677 -163.592  -216.7801 -206.825  -216.8559 -206.8178]
 [   0.        0.        0.       -0.        0.        0.        0.        0.     -125.6577 -125.7305 -129.9669 -129.9762 -199.8804 -199.786  -216.8518 -216.9001]
 [   0.        0.        0.        0.        0.        0.        0.        0.      -67.4078 -157.2906 -134.7205 -176.6425  -69.8961 -178.7498 -150.0812 -204.5439]
 [   0.        0.        0.        0.        0.        0.        0.        0.     -100.3094 -100.3124 -133.7057 -133.7202 -161.6872 -207.8942 -161.6895 -207.9013]
 [   0.        0.        0.        0.        0.        0.        0.        0.     -100.2994 -155.1654 -133.7072 -176.9053 -134.052  -201.149  -160.7825 -201.1573]
 [   0.        0.     

In [ ]:
'''Only State Information: Calculating Nash equilibria '''

epsilon = 0
# determinstic_strategy_itertools_no_info = itertools.product([0,1], repeat = 1)
determinstic_strategy_itertools_no_info = itertools.product([1,0], repeat = 1)

determinisic_strategy_lists_no_info = list(list(strat) for strat in determinstic_strategy_itertools_no_info)
all_determinstic_strategy_dictionary_no_info = {str(strat):strat for strat in determinisic_strategy_lists_no_info}
strategy_set_p1_no_info = all_determinstic_strategy_dictionary_no_info
strategy_set_p2_no_info = all_determinstic_strategy_dictionary_no_info

strategy_set_p1_both_only_state = {key: add_degraded_state_policies_only_state(value) for key, value in strategy_set_p1_no_info.items()}
strategy_set_p2_both_only_state = {key: add_degraded_state_policies_only_state(value) for key, value in strategy_set_p2_no_info.items()}

print(strategy_set_p1_both_only_state)

mode = 'ecological'
ecopg = BaseEcologicalPublicGood(m = -6)

information_condition_instance = Information_Conditions(ecopg, mode=mode)
mae_ecopg = POstratAC(env=information_condition_instance, learning_rates=0.05, discount_factors= 0.98)
p1_reward_matrix_only_state, p2_reward_matrix_only_state = metgame_reward_matrix_state(strategy_set_p1_both_only_state, strategy_set_p2_both_only_state,mode)

payoffs_row = p1_reward_matrix_only_state
payoffs_row_modifed = payoffs_row - payoffs_row[0,0]
print(payoffs_row)

payoffs_col  = p2_reward_matrix_only_state
print(payoffs_col)
game_only_state = pygambit.Game.from_arrays(payoffs_row, payoffs_col)


# Compute pure strategy Nash equilibria
result_only_state = pygambit.nash.enumpure_solve(game_only_state)
# result = pygambit.nash.enummixed_solve(game)

print(len(result_only_state.equilibria))
 
for i in result_only_state.equilibria:
    print(i)  
    print("---")

{'[1]': [0.03128370263515068, 1], '[0]': [0.625484167963071, 0]}
[[  50.     -298.0411]
 [-295.5898 -298.5377]]
[[  50.     -295.5915]
 [-298.0399 -298.5377]]
1
[[Rational(1, 1), Rational(0, 1)], [Rational(1, 1), Rational(0, 1)]]
---


In [ ]:
'''Only State Information: Calculating Nash equilibria '''

determinstic_strategy_itertools_no_info = itertools.product([1,0], repeat = 1)

determinisic_strategy_lists_no_info = list(list(strat) for strat in determinstic_strategy_itertools_no_info)
all_determinstic_strategy_dictionary_no_info = {str(strat):strat for strat in determinisic_strategy_lists_no_info}
strategy_set_p1_no_info = all_determinstic_strategy_dictionary_no_info
strategy_set_p2_no_info = all_determinstic_strategy_dictionary_no_info

strategy_set_p1_both_only_state = {key: add_degraded_state_policies_only_state(value) for key, value in strategy_set_p1_no_info.items()}
strategy_set_p2_both_only_state = {key: add_degraded_state_policies_only_state(value) for key, value in strategy_set_p2_no_info.items()}


print(strategy_set_p1_no_info)

# mode = 'only_state_information'
# ecopg = BaseEcologicalPublicGood()

mode = 'social'

env = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)



# information_condition_instance = Information_Conditions(ecopg, mode=mode)
# mae_ecopg = POstratAC(env=information_condition_instance, learning_rates=0.05, discount_factors= 0.98)
mae_socdi = stratAC(env=env, learning_rates=0.05, discount_factors= 0)

p1_reward_matrix_only_state, p2_reward_matrix_only_state = metgame_reward_matrix_state_test(strategy_set_p1_no_info, strategy_set_p2_no_info, mode,mae_socdi)

payoffs_row = p1_reward_matrix_only_state
# payoffs_row_modifed = payoffs_row - payoffs_row[0,0]
print(payoffs_row, "payoff row")

payoffs_col  = p2_reward_matrix_only_state

print(payoffs_col, "payoff col")

game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)


# Compute pure strategy Nash equilibria
result = pygambit.nash.enumpure_solve(game)
# result = pygambit.nash.enummixed_solve(game)

print(len(result.equilibria))
 
for i in result.equilibria:
    print(i)  
    print("---")




{'[1]': [1], '[0]': [0]}
[[ 0.8 -0.5]
 [ 1.   0. ]] payoff row
[[ 0.8  1. ]
 [-0.5  0. ]] payoff col
1
[[Rational(0, 1), Rational(1, 1)], [Rational(0, 1), Rational(1, 1)]]
---
